In [47]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [48]:
df_historical_clean = pd.read_csv('/Users/vinh/FS/thesis/data/df_historical_clean.csv')

In [49]:
df_historical_clean.loc[df_historical_clean['term'] == 36, 'term'] = 0
df_historical_clean.loc[df_historical_clean['term'] == 60, 'term'] = 1
df_historical_clean.loc[df_historical_clean['loan_status'] == 'Fully Paid', 'loan_status'] = 0
df_historical_clean.loc[df_historical_clean['loan_status'] == 'Charged Off', 'loan_status'] = 1

----
# Train/Validation/Test Split

In [50]:
X = df_historical_clean.drop('loan_status', axis = 1)
X['year'] = pd.to_datetime(X['issue_d']).dt.year
y = df_historical_clean[['loan_status']]

In [51]:
train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 1 - train_ratio, random_state = 1337, stratify = y)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size = test_ratio/(test_ratio + validation_ratio), random_state = 1337, stratify = y_test) 

#### ---- Ensure Correct Stratification ----
We want to stratify the state, year, and loan status to make sure that the train and test data have similar characteristics.

In [52]:
state_df = pd.DataFrame({'train': x_train[['addr_state']].value_counts(normalize = True),
                         'valid': x_val[['addr_state']].value_counts(normalize = True),
                         'test': x_test[['addr_state']].value_counts(normalize = True)})
state_df.sort_values(by = 'train', ascending = False).head()

,train,valid,test
addr_state,,,
CA,0.140187,0.141123,0.140579
TX,0.082652,0.082359,0.082480
NY,0.080466,0.080031,0.080254
FL,0.071537,0.071928,0.072405
IL,0.039019,0.039142,0.039259


In [53]:
year_df = pd.DataFrame({'train': x_train[['year']].value_counts(normalize = True),
                        'valid': x_val[['year']].value_counts(normalize = True),
                         'test': x_test[['year']].value_counts(normalize = True)})
year_df.sort_values(by = 'train', ascending = False).head()

,train,valid,test
year,,,
2016,0.228650,0.228927,0.228069
2015,0.228406,0.228617,0.228507
2017,0.178558,0.177006,0.178265
2014,0.133550,0.133982,0.133600
2018,0.111840,0.112216,0.111343


In [54]:
print(f"Train {y_train[['loan_status']].value_counts(normalize = True)}\n")
print(f"Val {y_val[['loan_status']].value_counts(normalize = True)}\n")
print(f"Test {y_test[['loan_status']].value_counts(normalize = True)}")

Train loan_status
0              0.8023
1              0.1977
dtype: float64

Val loan_status
0              0.802302
1              0.197698
dtype: float64

Test loan_status
0              0.802298
1              0.197702
dtype: float64


----
# Imputations

In [55]:
def na_values_df(df):
    '''
    Function which creates a table describing NA values.

    Arguments:
        df: pd.DataFrame
            Dataframe which you want to describe NA values for.

    Returns:
        Dataframe containing 3 columns which describes NA values in given df.
    '''
    na_val_count = df.isnull().sum()
    na_val_percent = df.isnull().sum() / len(df)
    na_val_type = df.dtypes

    na_val_df = pd.concat([na_val_count, na_val_percent, na_val_type], axis = 1)
    na_val_df = na_val_df.rename(columns = {0: 'na_value_count',
                                            1: 'na_value_percent',
                                            2: 'na_value_type'
    })
    na_val_df['na_value_percent'] = round(na_val_df['na_value_percent'], 5)
    na_val_df = na_val_df.sort_values(by = 'na_value_percent', ascending = False)
    
    return na_val_df

In [59]:
na_values_df(x_train)

,na_value_count,na_value_percent,na_value_type
emp_length,79624,0.06451,float64
revol_util,919,0.00074,float64
dti,776,0.00063,float64
id,0,0.00000,int64
pub_rec_bankruptcies,0,0.00000,float64
mort_acc,0,0.00000,float64
application_type,0,0.00000,object
initial_list_status,0,0.00000,object
revol_bal,0,0.00000,float64
pub_rec,0,0.00000,float64


In [60]:
imputations_df = pd.DataFrame({'emp_length_med': [x_train['emp_length'].median()],
                               'mort_acc_med': [x_train['mort_acc'].median()],
                               'revol_util_med': [x_train['revol_util'].median()],
                               'dti_med': [x_train['dti'].median()],
                               'pub_rec_bankruptcies_med': [x_train['pub_rec_bankruptcies'].median()]})

In [61]:
x_train.loc[x_train['emp_length'].isna(), 'emp_length'] = imputations_df['emp_length_med'].item()
x_train.loc[x_train['mort_acc'].isna(), 'mort_acc'] = imputations_df['mort_acc_med'].item()
x_train.loc[x_train['revol_util'].isna(), 'revol_util'] = imputations_df['revol_util_med'].item()
x_train.loc[x_train['dti'].isna(), 'dti'] = imputations_df['dti_med'].item()
x_train.loc[x_train['pub_rec_bankruptcies'].isna(), 'pub_rec_bankruptcies'] = imputations_df['pub_rec_bankruptcies_med'].item()

x_val.loc[x_val['emp_length'].isna(), 'emp_length'] = imputations_df['emp_length_med'].item()
x_val.loc[x_val['mort_acc'].isna(), 'mort_acc'] = imputations_df['mort_acc_med'].item()
x_val.loc[x_val['revol_util'].isna(), 'revol_util'] = imputations_df['revol_util_med'].item()
x_val.loc[x_val['dti'].isna(), 'dti'] = imputations_df['dti_med'].item()
x_val.loc[x_val['pub_rec_bankruptcies'].isna(), 'pub_rec_bankruptcies'] = imputations_df['pub_rec_bankruptcies_med'].item()

x_test.loc[x_test['emp_length'].isna(), 'emp_length'] = imputations_df['emp_length_med'].item()
x_test.loc[x_test['mort_acc'].isna(), 'mort_acc'] = imputations_df['mort_acc_med'].item()
x_test.loc[x_test['revol_util'].isna(), 'revol_util'] = imputations_df['revol_util_med'].item()
x_test.loc[x_test['dti'].isna(), 'dti'] = imputations_df['dti_med'].item()
x_test.loc[x_test['pub_rec_bankruptcies'].isna(), 'pub_rec_bankruptcies'] = imputations_df['pub_rec_bankruptcies_med'].item()

----
# Normalization

In [15]:
remove = ['id', 'issue_d', 'year', 'grade', 'sub_grade', 'term']

In [16]:
numerical_columns = list(x_train.select_dtypes(include = ['float64', 'int64']).columns)
numerical_columns = [x for x in numerical_columns if x not in remove]

In [17]:
scaler = MinMaxScaler()
x_train[numerical_columns] = scaler.fit_transform(x_train[numerical_columns])
x_val[numerical_columns] = scaler.transform(x_val[numerical_columns])
x_test[numerical_columns] = scaler.transform(x_test[numerical_columns])

----
# One Hot Encode

In [18]:
categorical_columns = list(x_train.select_dtypes(include = ['object']).columns)
categorical_columns = [x for x in categorical_columns if x not in remove]
categorical_columns.remove('initial_list_status')

transformers = [
    ('', OneHotEncoder(handle_unknown = 'ignore', sparse = False), categorical_columns)
]
ct = ColumnTransformer(transformers)

x_train_transformed = ct.fit_transform(x_train)
feature_names = ct.get_feature_names_out()
x_train_transformed = pd.DataFrame(x_train_transformed, columns = feature_names).set_index(x_train.index)
x_train_transformed.columns = x_train_transformed.columns.str[2:]

x_val_transformed = ct.transform(x_val)
x_val_transformed = pd.DataFrame(x_val_transformed, columns = feature_names).set_index(x_val.index)
x_val_transformed.columns = x_val_transformed.columns.str[2:]

x_test_transformed = ct.transform(x_test)
x_test_transformed = pd.DataFrame(x_test_transformed, columns = feature_names).set_index(x_test.index)
x_test_transformed.columns = x_test_transformed.columns.str[2:]

In [19]:
x_train_ohe = pd.concat([x_train[remove + numerical_columns], x_train_transformed], axis = 1)
x_val_ohe = pd.concat([x_val[remove + numerical_columns], x_val_transformed], axis = 1)
x_test_ohe = pd.concat([x_test[remove + numerical_columns], x_test_transformed], axis = 1)

----
# Save datasets for modeling

In [20]:
train_final = pd.concat([y_train, x_train_ohe], axis = 1)
val_final = pd.concat([y_val, x_val_ohe], axis = 1)
test_final = pd.concat([y_test, x_test_ohe], axis = 1)

train_final.to_csv('/Users/vinh/FS/thesis/data/train_final.csv', index = False)
val_final.to_csv('/Users/vinh/FS/thesis/data/val_final.csv', index = False)
test_final.to_csv('/Users/vinh/FS/thesis/data/test_final.csv', index = False)

----
# Create Balanced Data Set
From the train set, filter for defaults. Take 80% of the defaults. Take the same number of completed as you have defaults to create a balanced data set.

In [21]:
defaults = train_final.loc[train_final['loan_status'] == 1]
completed = train_final.loc[train_final['loan_status'] == 0]

In [22]:
defaults_train, defaults_test = train_test_split(defaults, test_size = 0.2, random_state = 1337)
completed_train, completed_test = train_test_split(completed,
                                                   test_size = defaults_train.shape[0] / completed.shape[0],
                                                   random_state = 1337)

In [23]:
train_balanced = pd.concat([completed_test, defaults_train])
train_balanced['loan_status'].value_counts()

0    195220
1    195220
Name: loan_status, dtype: int64

In [24]:
train_balanced.to_csv('/Users/vinh/FS/thesis/data/train_balanced.csv', index = False)